<a href="https://colab.research.google.com/github/cyclic-reference/fast-ai-swift/blob/master/initial_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%install-location $cwd/swift-install
%install '.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")' Path
%install '.package(url: "https://github.com/saeta/Just", from: "0.7.2")' Just
%install '.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")' NotebookExport

Installing packages:
	.package(url: "https://github.com/mxcl/Path.swift", from: "0.16.1")
		Path
	.package(url: "https://github.com/saeta/Just", from: "0.7.2")
		Just
	.package(url: "https://github.com/latenitesoft/NotebookExport", from: "0.5.0")
		NotebookExport
With SwiftPM flags: []
Working in: /tmp/tmphm__rrxh/swift-install
[1/2] Compiling jupyterInstalledPackages jupyterInstalledPackages.swift
[2/3] Merging module jupyterInstalledPackages
Initializing Swift...
Installation complete!


In [0]:
// export
precedencegroup ExponentiationPrecedence {
    associativity: right
    higherThan: MultiplicationPrecedence
}
infix operator ** : ExponentiationPrecedence

precedencegroup CompositionPrecedence { associativity: left }
infix operator >| : CompositionPrecedence

In [0]:
//export
import Foundation
import Just
import Path

In [0]:
//export
public extension String {
    @discardableResult
    func shell(_ args: String...) -> String {
        let (task,pipe) = (Process(),Pipe())
        task.executableURL = URL(fileURLWithPath: self)
        (task.arguments,task.standardOutput) = (args,pipe)
        do    { try task.run() }
        catch { print("Unexpected error: \(error).") }

        let data = pipe.fileHandleForReading.readDataToEndOfFile()
        return String(data: data, encoding: String.Encoding.utf8) ?? ""
    }
}

In [21]:
print("/bin/ls".shell("-lh"))

total 9.5M
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data
drwxr-xr-x 4 root root 4.0K Sep  4 22:16 swift-install
-rw------- 1 root root 9.5M Sep  4 22:20 train-images-idx3-ubyte.gz



In [0]:
//export
public func ll(){
  print("/bin/ls".shell("-lh"))
}

In [23]:
ll()

total 9.5M
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data
drwxr-xr-x 4 root root 4.0K Sep  4 22:16 swift-install
-rw------- 1 root root 9.5M Sep  4 22:20 train-images-idx3-ubyte.gz



In [0]:
//export
public func downloadFile(_ url: String, dest: String? = nil, force: Bool = false) {
    let dest_name = dest ?? (Path.cwd/url.split(separator: "/").last!).string
    let url_dest = URL(fileURLWithPath: (dest ?? (Path.cwd/url.split(separator: "/").last!).string))
    if !force && Path(dest_name)!.exists { return }

    print("Downloading \(url)...")

    if let cts = Just.get(url).content {
        do    {try cts.write(to: URL(fileURLWithPath:dest_name))}
        catch {print("Can't write to \(url_dest).\n\(error)")}
    } else {
        print("Can't reach \(url)")
    }
}

In [0]:
downloadFile("https://storage.googleapis.com/cvdf-datasets/mnist/train-images-idx3-ubyte.gz")

In [0]:
//export
import TensorFlow

In [0]:
//export
protocol ConvertibleFromByte: TensorFlowScalar {
    init(_ d:UInt8)
}

In [0]:
//export
extension Float : ConvertibleFromByte {}
extension Int32 : ConvertibleFromByte {}

In [0]:
//export
extension Data {
    func asTensor<T:ConvertibleFromByte>() -> Tensor<T> {
        return Tensor(map(T.init))
    }
}

In [0]:
//export
func loadMNIST<T: ConvertibleFromByte>
            (training: Bool, labels: Bool, path: Path, flat: Bool) -> Tensor<T> {
    let split = training ? "train" : "t10k"
    let kind = labels ? "labels" : "images"
    let batch = training ? 60000 : 10000
    let shape: TensorShape = labels ? [batch] : (flat ? [batch, 784] : [batch, 28, 28])
    let dropK = labels ? 8 : 16
    let baseUrl = "https://storage.googleapis.com/cvdf-datasets/mnist/"
    let fname = split + "-" + kind + "-idx\(labels ? 1 : 3)-ubyte"
    let file = path/fname
    if !file.exists {
        downloadFile("\(baseUrl)\(fname).gz", dest:(path/"\(fname).gz").string)
        "/bin/gunzip".shell("-fq", (path/"\(fname).gz").string)
    }
    let data = try! Data(contentsOf: URL(fileURLWithPath: file.string)).dropFirst(dropK)
    if labels { return data.asTensor() }
    else      { return data.asTensor().reshaped(to: shape)}
}

public func loadMNIST(path:Path, flat:Bool = false)
        -> (Tensor<Float>, Tensor<Int32>, Tensor<Float>, Tensor<Int32>) {
    try! path.mkdir(.p)
    return (
        loadMNIST(training: true,  labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: true,  labels: true,  path: path, flat: flat),
        loadMNIST(training: false, labels: false, path: path, flat: flat) / 255.0,
        loadMNIST(training: false, labels: true,  path: path, flat: flat)
    )
}

In [0]:
public let mnistPath = Path.cwd

In [32]:
let (xTrain, yTrain, xValid, yValid) = loadMNIST(path: mnistPath)
xTrain.shape

▿ [60000, 28, 28]
  ▿ dimensions : 3 elements
    - 0 : 60000
    - 1 : 28
    - 2 : 28


In [33]:
ll()

total 53M
drwxr-xr-x 1 root root 4.0K Aug 27 16:17 sample_data
drwxr-xr-x 4 root root 4.0K Sep  4 22:16 swift-install
-rw------- 1 root root 7.5M Sep  4 22:49 t10k-images-idx3-ubyte
-rw------- 1 root root 9.8K Sep  4 22:49 t10k-labels-idx1-ubyte
-rw------- 1 root root  45M Sep  4 22:20 train-images-idx3-ubyte
-rw------- 1 root root  59K Sep  4 22:49 train-labels-idx1-ubyte



In [34]:
"/swift/toolchain/usr/bin/swift".shell("--version")

"Swift version 5.1-dev (LLVM af1f73e9e9, Swift 7d157f346b)\nTarget: x86_64-unknown-linux-gnu\n"


In [0]:
//export 
import Dispatch

// ⏰Time how long it takes to run the specified function, optionally taking
// the average across a number of repetitions.
public func time(repeating: Int = 1, _ f: () -> ()) {
    guard repeating > 0 else { return }
    
    // Warmup
    if repeating > 1 { f() }
    
    var times = [Double]()
    for _ in 1...repeating {
        let start = DispatchTime.now()
        f()
        let end = DispatchTime.now()
        let nanoseconds = Double(end.uptimeNanoseconds - start.uptimeNanoseconds)
        let milliseconds = nanoseconds / 1e6
        times.append(milliseconds)
    }
    print("average: \(times.reduce(0.0, +)/Double(times.count)) ms,   " +
          "min: \(times.reduce(times[0], min)) ms,   " +
          "max: \(times.reduce(times[0], max)) ms")
}

In [39]:
time(repeating:10) {
  for thing in 1...100 {
      print(".","\r ")
  }
}

In [0]:
// export
public extension String {
    func findFirst(pat: String) -> Range<String.Index>? {
        return range(of: pat, options: .regularExpression)
    }
    func hasMatch(pat: String) -> Bool {
        return findFirst(pat:pat) != nil
    }
}

In [0]:
//export
public func notebookToScript(fname: Path){
    let newname = fname.basename(dropExtension: true)+".swift"
    let url = fname.parent/"FastaiNotebooks/Sources/FastaiNotebooks"/newname
    do {
        let data = try Data(contentsOf: fname.url)
        let jsonData = try JSONSerialization.jsonObject(with: data, options: .allowFragments) as! [String: Any]
        let cells = jsonData["cells"] as! [[String:Any]]
        var module = """
/*
THIS FILE WAS AUTOGENERATED! DO NOT EDIT!
file to edit: \(fname.lastPathComponent)

*/
        
"""
        for cell in cells {
            if let source = cell["source"] as? [String], !source.isEmpty, 
                   source[0].hasMatch(pat: #"^\s*//\s*export\s*$"#) {
                module.append("\n" + source[1...].joined() + "\n")
            }
        }
        try module.write(to: url, encoding: .utf8)
    } catch {
        print("Can't read the content of \(fname)")
    }
}

In [0]:
// export
public func exportNotebooks(_ path: Path) {
    for entry in try! path.ls()
    where entry.kind == Entry.Kind.file && 
          entry.path.basename().hasMatch(pat: #"^\d*_.*ipynb$"#) {
        print("Converting \(entry)")
        notebookToScript(fname: entry.path)
    }
}

In [0]:
import NotebookExport
let exporter = NotebookExport(Path.cwd/".ipynb")
print(exporter.export(usingPrefix: "FastaiNotebook_"))

In [51]:
print("/bin/ls".shell("/datalab"))

run.sh
web



In [53]:
Path.cwd.basename()

"content"
